In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import keras
import tensorflow as tf
from sklearn.metrics import f1_score
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Dropout, Conv2D

In [ ]:
import tensorflow.keras.backend as K
train_datagen = ImageDataGenerator(rescale=1./255, width_shift_range=0.1, height_shift_range=0.1)
path = '/content/drive/MyDrive/Machine_Learning/Data/KyTu'
train_generator = train_datagen.flow_from_directory(
        path+'/train',
        target_size=(28,28),
        batch_size=1,
        class_mode='sparse')

validation_generator = train_datagen.flow_from_directory(
        path+'/val',
        target_size=(28,28),
        class_mode='sparse')

Found 4964 images belonging to 31 classes.
Found 906 images belonging to 31 classes.


In [ ]:
def f1score(y, y_pred):
  return f1_score(y, tf.math.argmax(y_pred, axis=1), average='micro')

def custom_f1score(y, y_pred):
  return tf.py_function(f1score, (y, y_pred), tf.double)

In [ ]:
K.clear_session()
model = Sequential()
model.add(Conv2D(16, (22,22), input_shape=(28, 28, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (16,16), input_shape=(28, 28, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (8,8), input_shape=(28, 28, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (4,4), input_shape=(28, 28, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(31, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0001), metrics=[custom_f1score])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 16)        23248     
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        131104    
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        131136    
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 64)        65600     
                                                                 
 max_pooling2d (MaxPooling2  (None, 7, 7, 64)          0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 7, 7, 64)          0         
                                                        

In [ ]:
class stop_training_callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_custom_f1score') > 0.99):
      self.model.stop_training = True

In [ ]:
batch_size = 1
callbacks = [stop_training_callback()]
model.fit_generator(
      train_generator,
      steps_per_epoch = train_generator.samples // batch_size,
      validation_data = validation_generator,
      epochs = 10, verbose=1, callbacks=callbacks)

<ipython-input-9-214927322d25>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
4964/4964 [==============================] - 1377s 277ms/step - loss: 0.6607 - custom_f1score: 0.8396 - val_loss: 0.3282 - val_custom_f1score: 0.9203
Epoch 2/10
4964/4964 [==============================] - 525s 106ms/step - loss: 0.2222 - custom_f1score: 0.9434 - val_loss: 0.2425 - val_custom_f1score: 0.9457
Epoch 3/10
4964/4964 [==============================] - 520s 105ms/step - loss: 0.1722 - custom_f1score: 0.9575 - val_loss: 0.1953 - val_custom_f1score: 0.9580
Epoch 4/10
4964/4964 [==============================] - 523s 105ms/step - loss: 0.1392 - custom_f1score: 0.9664 - val_loss: 0.1280 - val_custom_f1score: 0.9655
Epoch 5/10
4964/4964 [==============================] - 525s 106ms/step - loss: 0.1158 - custom_f1score: 0.9702 - val_loss: 0.1130 - val_custom_f1score: 0.9709
Epoch 6/10
4964/4964 [==============================] - 529s 106ms/step - loss: 0.1075 - custom_f1score: 0.9762 - val_loss: 0.0847 - val_custom_f1score: 0.9806
Epoch 7/10
4964/4964 [=================

In [ ]:
model.save('/content/drive/MyDrive/Machine_Learning/Model_KyTu.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model.save('/content/drive/MyDrive/Machine_Learning/Model_KyTu.keras')